In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from data_preprocessing import image_preprocessing
from model import build_model
slim = tf.contrib.slim

checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'
label_file = 'training/label.idl'
train_file = 'training/'
validation_file = 'validation/'

In [4]:
def train_graph(model_list, image_preprocessor):
    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    saver, images, box_confidence, box_class_label, box_coordinate, train_op, recall, precise, loss, confidence = model_list

    with tf.Session(config=config) as sess:
        sess.run(init)
        saver.restore(sess, checkpoint_file)
        for i in range(30000):
            the_images, labels = image_preprocessor.get_next_image_batch(128)
            sess.run([train_op], feed_dict={images: the_images, 
                                         box_confidence: labels['confidence'],
                                         box_class_label: labels['class_label'],
                                         box_coordinate: labels['coordinate']})

    #         sum_weights = np.sum(test_weights)
    #         the_loss = sess.run(train_op, feed_dict={images: the_images, 
    #                                                  box_confidence: labels['confidence'],
    #                                                  box_class_label: labels['class_label'],
    #                                                  box_coordinate: labels['coordinate']})
            if(i % 100 == 0):
                the_confidence, validation_loss, validation_recall, validation_precise = sess.run([confidence,loss, recall, precise], feed_dict={images:image_preprocessor.validation_images,
                                                            box_confidence: image_preprocessor.validation_labels['confidence'],
                                                            box_class_label: image_preprocessor.validation_labels['class_label'],
                                                            box_coordinate: image_preprocessor.validation_labels['coordinate']})
                
                print('###############################################################')
                print('the loss after ' + str(i) + ' iteration is: ' + str(the_loss))
                print('the recall after ' + str(i) + ' iteration is: ' + str(the_recall))
                print('the precise after ' + str(i) + ' iteration is: ' + str(the_precise))
                print('*****')
                print('the validation loss after ' + str(i) + ' iteration is: ' + str(validation_loss))
                print('the validation recall after ' + str(i) + ' iteration is: ' + str(validation_recall))
                print('the validation precise after ' + str(i) + ' iteration is: ' + str(validation_precise))
                print(np.mean(the_confidence))
                print(np.amax(the_confidence))



In [5]:
def train_small_data(model_list, image_processor):
    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    saver, images, box_confidence, box_class_label, box_coordinate, train_op, recall, precise, loss, confidence = model_list
    
    the_images = image_preprocessor.validation_images[:2,:,:,:]
    the_labels_conf = image_preprocessor.validation_labels['confidence'][:2,:,:,:]
    the_labels_coor = image_preprocessor.validation_labels['coordinate'][:2,:,:,:]
    the_labels_class = image_preprocessor.validation_labels['class_label'][:2,:,:,:]

    with tf.Session(config=config) as sess:
        sess.run(init)
        saver.restore(sess, checkpoint_file)
        for i in range(30000):
            sess.run([train_op],
                     feed_dict={images: the_images, 
                     box_confidence: the_labels_conf,
                     box_class_label: the_labels_class,
                     box_coordinate: the_labels_coor})
            
            if(i % 100 == 0):
                the_confidence, the_loss, the_recall, the_precise = sess.run([confidence, loss, recall, precise],
                                                                             feed_dict={images: the_images, 
                                                                             box_confidence: the_labels_conf,
                                                                             box_class_label: the_labels_class,
                                                                             box_coordinate: the_labels_coor})
                print('###############################################################')
                print('the loss after ' + str(i) + ' iteration is: ' + str(the_loss))
                print('the recall after ' + str(i) + ' iteration is: ' + str(the_recall))
                print('the precise after ' + str(i) + ' iteration is: ' + str(the_precise))
                print(np.mean(the_confidence))
                print(np.amax(the_confidence))



In [6]:
# build model
model_list = build_model()

/usr/lib/python3.4/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
# get image processor
image_preprocessor = image_preprocessing(train_file, label_file, validation_file)
image_preprocessor.prepare_data()

In [8]:
# start training
train_small_data(model_list, image_preprocessor)

INFO:tensorflow:Restoring parameters from ./inception_resnet_v2_2016_08_30.ckpt
###############################################################
the loss after 0 iteration is: 25.0
the recall after 0 iteration is: 0.0
the precise after 0 iteration is: nan
0.0
0.0
###############################################################
the loss after 100 iteration is: 25.0
the recall after 100 iteration is: 0.0
the precise after 100 iteration is: nan
0.0
0.0
###############################################################
the loss after 200 iteration is: 25.0
the recall after 200 iteration is: 0.0
the precise after 200 iteration is: nan
0.0
0.0
###############################################################
the loss after 300 iteration is: 25.0
the recall after 300 iteration is: 0.0
the precise after 300 iteration is: nan
0.0
0.0
###############################################################
the loss after 400 iteration is: 25.0
the recall after 400 iteration is: 0.0
the precise after 400 iterati

KeyboardInterrupt: 